In [1]:
import sys
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#open file with history
path = 'msg/messages.html'
with open(path, encoding='utf8') as file:
    data = file.read()

In [3]:
#creating parser
soup = BeautifulSoup(data, 'html.parser')

In [4]:
#looking for author name
channel_name = soup.find_all('div', attrs={'class':'page_header'})
channel_name = channel_name[0].find('div', attrs={'class':'text bold'}).text[1:-7]

In [5]:
#get all messages in block 'history'
history = soup.find_all('div', attrs={'class':'history'})
history = history[0]
messages = history.find_all('div', attrs={'class':'message default clearfix'})
len(messages)

998

In [6]:
EMPTY_TEXT = '_empty_text_'

#creating DataFrame with messages
records = [] # temp operand 

#for every message in history
for msg in messages:
    #all messages have date
    date = msg.find('div', attrs={'class':'pull_right date details'})['title']
    
    #if message is forwarded
    if len(msg.find_all('div', attrs={'class':'forwarded body'})) != 0: 
        #get message
        forwarded = msg.find('div', attrs={'class':'forwarded body'})
        #find autor, get out as text and correct string lenght
        source = forwarded.find('div', attrs={'class':'from_name'}).text[1:-22]
        #get text block
        text = msg.find('div', attrs={'class':'text'}) 
        #if message has text get it out and correct string lenght
        if  text is not None:
            text = text.text[1:-8]
        msg_type = 'forwarded'
        autor = source
    
    #if message is a poll
    elif len(msg.find_all('div', attrs={'class':'media_poll'})) != 0:
        #get poll
        poll = msg.find('div', attrs={'class':'media_poll'})
        #get qustion
        question = poll.find('div', attrs={'class':'question bold'}).text[1:-10]
        #get tuple of answers
        answers = poll.find_all('div', attrs={'class':'answer'})
        #building string in format "qustion: answer, answer..."
        text = question + ': '
        for answer in answers:
            text += answer.text[3:-10] + ', '
        text = text[:-2]
        
        msg_type = 'poll'
        autor = channel_name
    
    #else some sort of message
    else: 
        #get text block
        text = msg.find('div', attrs={'class':'text'}) 
        #if message has text get it out and correct string lenght
        if  text is not None:
            text = text.text[1:-8]
        msg_type = 'message'
        autor = channel_name
        
        
    #print(date, msg_type, autor)    
    records.append((date, msg_type, autor, text))
df = pd.DataFrame(records,columns=['date', 'type', 'autor', 'text'])
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
#df.sort_index(inplace=True) # not recomended, forwarded messages might be older then previous 
df['text'].fillna(EMPTY_TEXT, inplace=True) #some messageses may not has text, so fill them with well known constant
#show first 10 
df.head(10)

,type,autor,text
date,,,
2016-04-07 12:15:06,message,Чай з малинавым варэннем,В Минске праздничный потопЧитать
2016-04-07 12:15:07,message,Чай з малинавым варэннем,10 фото
2016-04-07 12:38:06,message,Чай з малинавым варэннем,"Мы тут канал в Telegram завели, го:https://tel..."
2016-04-07 12:38:07,message,Чай з малинавым варэннем,_empty_text_
2016-04-07 13:27:06,message,Чай з малинавым варэннем,"История про то, как у главного дорожника Минск..."
2016-04-07 13:27:07,message,Чай з малинавым варэннем,_empty_text_
2016-04-07 14:35:05,message,Чай з малинавым варэннем,"В Орше вор украл велосипед, перекрасил его и п..."
2016-04-07 20:02:08,message,Чай з малинавым варэннем,_empty_text_
2016-05-07 10:31:06,message,Чай з малинавым варэннем,В Гродно столкнулись две милицейские машиныhtt...


In [7]:
df.iloc[1]['text']

'10 фото'